### Install the required packages

In [ ]:
# %pip install python-dotenv
# %pip install requests
# %pip install azure-ai-inference
# %pip install --upgrade python-certifi-win32
# %pip install --upgrade certifi
# %pip install pip-system-certs

### Load the configuration settings

In [ ]:
from dotenv import dotenv_values,load_dotenv
from azure.ai.inference import ChatCompletionsClient, EmbeddingsClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference.models import SystemMessage, UserMessage
import sys
sys.path.append('../common')
from helper import Helper
import os

env = os.getenv('GITHUB_ENV') or 'local'
env_name = os.getenv('ENV_NAME') or 'GO-AMER-USE-DV'

load_dotenv()

secrets = dotenv_values("secrets.env")
api_key = secrets.get("API_KEY")
environment = secrets.get("APIM_NAME")
region_override = secrets.get("REGION_OVERRIDE")
helper = Helper(environment, "genai/azure/deployments", api_key, region_override)

helper.log('GITHUB_ENV: ' + env)
helper.log('ENV_NAME: ' + env_name)

### Get Deployment IDs List

In [ ]:
env_config = helper.read_json_config(env_name)

# Check if deployment_info is found
assert env_config is not None, f"Deployment information not found for environment: {env_name}"

chat_model_deployments_list = env_config["api_configuration"]["inference"]['chat_models']

flattened_chat_model_deployments_list = []
for deployment in chat_model_deployments_list:
    default_api_version = deployment["api_version"]
    for model in deployment["models"]:
        # Special handling for o3-mini models to ensure they use 2024-12-01-preview or later
        api_version = default_api_version
        if ("o3-mini" in model) or ("o1" in model):
            # Ensure o3-mini and o1 models use 2024-12-01-preview or later
            api_version = "2024-12-01-preview"
        flattened_chat_model_deployments_list.append({"api_version": api_version, "model": model})

helper.log("Chat Models:")
helper.log(chat_model_deployments_list)
helper.log(flattened_chat_model_deployments_list)

### No Charge Header

In [ ]:
# Create a client
client = ChatCompletionsClient(
    endpoint=f"https://{environment}/genai/azure/inference",
    credential=AzureKeyCredential(""),
    api_version=flattened_chat_model_deployments_list[0]['api_version'],
    headers={
        "Ocp-Apim-Subscription-Key": api_key,
        "azureml-model-deployment": flattened_chat_model_deployments_list[0]['model'],
        "x-kpmg-region-override": region_override,
        # remove x-kpmg-charge-code header and test if the response will fail
    },
)

try:
    # Create a completion request
    _ = client.complete(
        messages=[
            SystemMessage(content="You are a helpful assistant."),
            UserMessage(
                content="Tell me a bit about your your creator, and the model that you run on?"
            ),
        ]
    )
except Exception as err:
    error = err
    helper.log(error)

# Validate the response
assert error.status_code == 400
assert "missing the required 'x-kpmg-charge-code'" in error.message

### Chat Completion (Non-Streaming)

In [ ]:
import concurrent.futures

def test_deployment(deployment):
    try:
        client = ChatCompletionsClient(
            endpoint=f"https://{environment}/genai/azure/inference",
            credential=AzureKeyCredential(api_key),
            api_version=deployment['api_version'],
            headers={
                "Ocp-Apim-Subscription-Key": api_key,
                'x-kpmg-charge-code': '803000015283',
                "azureml-model-deployment": deployment['model'],
                "x-kpmg-region-override": region_override,
            }
        )

        # Create a completion request
        response = client.complete(
            messages=[
                SystemMessage(content="You are a helpful assistant."),
                UserMessage(content="What is the capital of France?"),
            ]
        )

        # Output the completion response
        helper.log(response.choices[0].message.content)

        # Validate the response
        assert len(response.choices) == 1
        assert len(response.choices[0].message.content) > 0
        helper.log(f"++ Deployment_ID:{deployment['model']} Succeeded ++")
        helper.log('')
        return True
    except Exception as err:
        try:
            helper.log(f"XX Deployment_ID:{deployment['model']} failed XX, Error: {str(err)}")
        except UnicodeEncodeError:
            helper.log(f"XX Deployment ID failed XX, Error: Unicode encoding error with model name")
        return False

success_count = 0
failure_count = 0

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(test_deployment, flattened_chat_model_deployments_list))

success_count = results.count(True)
failure_count = results.count(False)

helper.log(f"Total Success: {success_count}, Total Failure: {failure_count} of {len(flattened_chat_model_deployments_list)}")

assert failure_count == 0, f'{failure_count} of {len(flattened_chat_model_deployments_list)} deployments failed'


### Chat Completions (Streaming)

In [ ]:
import concurrent.futures

def test_deployment_streaming(deployment):
    try:
        # Create a client
        client = ChatCompletionsClient(
            endpoint=f"https://{environment}/genai/azure/inference",
            credential=AzureKeyCredential(api_key),
            api_version=deployment['api_version'],
            headers={
                "Ocp-Apim-Subscription-Key": api_key,
                'x-kpmg-charge-code': '803000015283',
                "azureml-model-deployment": deployment['model'],
                "x-kpmg-region-override": region_override,
            }
        )

        # Create a completion request
        response = client.complete(
            stream=True,
            messages=[
                SystemMessage(content="You are a helpful assistant."),
                UserMessage(content="What is the capital of France?"),
            ]
        )

        parts = []

        # Output the completion response
        for chunk in response:
            if len(chunk.choices) > 0:
                delta = chunk.choices[0].delta

                if delta.content:
                    helper.log(delta.content, '')
                    parts.append(delta.content)

        helper.log('')

        # Validate the response
        assert len(parts) > 0

        helper.log(f"++ Deployment_ID:{deployment['model']} Succeeded ++")
        helper.log('')

        return True
    except Exception as err:
        try:
            helper.log(f"XX Deployment_ID:{deployment['model']} failed XX, Error: {str(err)}")
        except UnicodeEncodeError:
            helper.log(f"XX Deployment ID failed XX, Error: Unicode encoding error with model name")
        return False

success_count = 0
failure_count = 0

# Use the already created flattened list with the correct API versions
# No need to re-flatten here as we'll use the same list from above

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(test_deployment_streaming, flattened_chat_model_deployments_list))

success_count = results.count(True)
failure_count = results.count(False)

helper.log(f"Total Success: {success_count}, Total Failure: {failure_count} of {len(flattened_chat_model_deployments_list)}")

assert failure_count == 0, f'{failure_count} of {len(flattened_chat_model_deployments_list)} deployments failed'

### Embedding

In [ ]:
embeddings_model_deployment = env_config["api_configuration"]["inference"]['embeddings_models'][0]

helper.log("Embeddings Models:")
helper.log(embeddings_model_deployment)

embeddings_deployment = embeddings_model_deployment["name"]
expected_length = embeddings_model_deployment["dimensions"]

# Create a client
client = EmbeddingsClient(
    endpoint=f"https://{environment}/genai/azure/inference",
    credential=AzureKeyCredential(""),
    headers={
        "Ocp-Apim-Subscription-Key": api_key,
        "azureml-model-deployment": embeddings_deployment,
        "x-kpmg-charge-code": "123123",  # random charge code
        "x-kpmg-region-override": region_override,
        "content-type": "application/json",
    },
)

# Create an embedding request
response = client.embed(input=["Hi!, how can you be of any assistance?"])
embedding = response["data"][0].embedding
helper.log(embedding)

assert len(embedding) == expected_length
assert any(embedding)  # check if there is any zero values

for value in embedding:
    assert isinstance(value, float)  # check if the embeddings are float
